# <center>Quickstart Guide</center>
## <center>Gain insights into your model via Phoenix</center>

Phoenix first and foremost is an application that can run alongside your notebook environment. It takes in up to two sets of data and surfaces up drift, performance, and data quality insights.


### 📚 Install `arize-phoenix` 

In [ ]:
%pip install -q arize-phoenix

### Using a built-in dataset to view the application

To get familiar with the application itself, the easiest way to get started is to use one of phoenix's fixture datasets.

In [ ]:
import phoenix as px

# Get the fixture datasets via the fixture name. Some valid values are "fashion_mnist", "sentiment_classification_language_drift", and "credit_card_fraud"
datasets = px.load_datasets("sentiment_classification_language_drift")
session = px.launch_app(datasets.primary, datasets.reference)
session.view()
